In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import glob
import re
from matplotlib import colors
import collections
import matplotlib
import scipy.stats as stats
import math
import lmfit
import pandas as pd

#for event display
import sys
#path to event display directory - change with where your path is 
# sys.path.append("C:/Users/ms2609/Downloads/WCTE_event_display-main/")
import EventDisplay
import matplotlib.colors as colors
from platform import python_version
print(python_version())

3.11.5


## Load in data

Loads in the data, break in the loop to only load the first 10 files - can be tuned to look at specific files. Notebook will crash if too much data is loaded - loading only hits and not waveforms is an option to reduce memory

In [2]:
runNumber = 1738
#f'../data/2025_LaserBall/WCTE_offline_R{run}S*P*.root'
file_pattern = "../WCTE_offline_R"+str(runNumber)+"S0P*.root"
file_list = glob.glob(file_pattern)
print (file_list)

def extract_p_number(file_name):
    match = re.search(r'P(\d+)\.root$', file_name)  # Extract number after 'P'
    return int(match.group(1)) if match else float('inf')  # Convert to int, default high if no match

file_list = sorted(file_list, key=extract_p_number)
# print(file_list)

all_events_list = []

# Loop over files and load data
for ifile, file_path in enumerate(file_list):
    if ifile>9:
        break
    print(f"Loading: {file_path}")
    
    with uproot.open(file_path) as root_file:
        tree = root_file["WCTEReadoutWindows"]
        events = tree.arrays(library="ak")  # Load branches as awkward arrays
        all_events_list.append(events)  # Store in list

# Concatenate everything into a single awkward array
if all_events_list:
    all_events = ak.concatenate(all_events_list,axis=0)
    
print(f"Total events loaded: {len(all_events)}")

['..\\WCTE_offline_R1738S0P12.root']
Loading: ..\WCTE_offline_R1738S0P12.root
Total events loaded: 56438


In [3]:
def get_run(infiles):
    files = infiles
    n_windows = 0
    n_hits = 0
    for i, f in enumerate(files):
        tree = uproot.open(f+":WCTEReadoutWindows")
        n_windows += tree.num_entries
        n_hits += ak.count(tree['hit_mpmt_card_ids'].array())
    window_times = np.zeros(n_windows, dtype=np.float64)
    event_numbers = np.zeros(n_windows, dtype=np.int32)
    window_part_files = np.zeros(n_windows, dtype=np.int16)
    hit_mpmt_card_ids = np.zeros(n_hits, dtype=np.int16)
    hit_pmt_channel_ids = np.zeros(n_hits, dtype=np.int8)
    hit_pmt_times = np.memmap("C:/Users/ms2609/Downloads/hit_pmt_times.memmap", dtype=np.float64, mode='w+', shape=n_hits)
    hit_pmt_charges = np.memmap("C:/Users/ms2609/Downloads/hit_pmt_charges.memmap", dtype=np.float32, mode='w+', shape=n_hits)
    hit_part_files = np.zeros(n_hits, dtype=np.int16)
    max_event_number = -1
    event_count = 0
    hit_count = 0
    for i, f in enumerate(files):
        print(f"  loading data from file {i}: {f}")
        tree = uproot.open(f+":WCTEReadoutWindows")
        
        next_event_count = event_count + tree.num_entries
        window_part_files[event_count:next_event_count] = i
        file_window_times = tree['window_time'].array()
        window_times[event_count:next_event_count] = file_window_times
        event_numbers[event_count:next_event_count] = tree['event_number'].array() + max_event_number + 1
        max_event_number = np.max(event_numbers[:next_event_count])

        card_ids = ak.flatten(tree['hit_mpmt_card_ids'].array())
        next_hit_count = hit_count+len(card_ids)
        hit_mpmt_card_ids[hit_count:next_hit_count] = card_ids
        hit_pmt_channel_ids[hit_count:next_hit_count] = ak.flatten(tree['hit_pmt_channel_ids'].array())
        hit_pmt_times[hit_count:next_hit_count] = ak.flatten(tree['hit_pmt_times'].array()+file_window_times)
        hit_pmt_charges[hit_count:next_hit_count] = ak.flatten(tree['hit_pmt_charges'].array())
        hit_part_files[hit_count:next_hit_count] = i

        event_count = next_event_count
        hit_count = next_hit_count

    return hit_pmt_times, hit_pmt_charges, hit_mpmt_card_ids, hit_pmt_channel_ids, hit_part_files, window_times, event_numbers, window_part_files

In [4]:
def dataQuality():
    max_good_t = np.max(window_times[window_times<np.quantile(window_times, 0.99)*1.5])+1
    good_times = hit_pmt_times < max_good_t
    good_card_ids = (hit_mpmt_card_ids > 0) & (hit_mpmt_card_ids < 130) & (hit_mpmt_card_ids !=106)
    good = good_times & good_card_ids
    trigger_hits = (hit_mpmt_card_ids==131)&(hit_pmt_channel_ids==5) & good_times
    
    return good, trigger_hits

In [5]:
hit_pmt_times, hit_pmt_charges, hit_mpmt_card_ids, hit_pmt_channel_ids, hit_part_files, window_times, event_numbers, window_part_files = get_run(file_list)
good, trigger_hits = dataQuality()

  loading data from file 0: ..\WCTE_offline_R1738S0P12.root


In [6]:
def gaussian(x,Qn,sigman):
    norm = np.sqrt(2/np.pi)/(sigman*(1+np.math.erf(Qn / (np.sqrt(2) * sigman))))
    return norm*np.exp(-(x-Qn)**2/(2*sigman**2))

def poisson_term(mu,n):
    return mu**n * np.exp(-mu)/np.vectorize(np.math.factorial)(n)

def pmtresponse(xx,N, Q1, s1, mu, w, alpha):
    x = xx
    
    signal = 0
    for n in range(1,3):
        Qn = n*Q1
        sigman = np.sqrt(n)*s1
        poisson = poisson_term(mu,n)
        gauss_term = gaussian(x,Qn,sigman)
        exp_term = alpha/2 * np.exp(-alpha * (x - Qn - alpha*sigman**2/2))
        erf1 = np.math.erf(abs(n*Q1 + sigman*sigman*alpha)/(sigman*np.sqrt(2)))
        erfarg = x - Qn - sigman*sigman*alpha
        erf2 = np.sign(erfarg)*np.vectorize(np.math.erf)(abs(erfarg)/(sigman*np.sqrt(2)))
        IGn_term = exp_term*(erf1 + erf2)
        signal += poisson*((1-w)*gauss_term + w*IGn_term)
    return N*signal

def pmtbackgroundresponse(xx,N, Q1, s1, mu, w, alpha):
    x = xx
    # N, Q1, s1, mu, w, alpha = p
    
    signal = 0
    for n in range(1,3):
        Qn = n*Q1
        sigman = np.sqrt(n)*s1
        poisson = poisson_term(mu,n)
        gauss_term = gaussian(x,Qn,sigman)
        erfarg = x - Qn - sigman*sigman*alpha
        exp_term = alpha/2 * np.exp(-alpha * erfarg)
        erf1 = np.math.erf(abs(Qn + sigman*sigman*alpha)/(sigman*np.sqrt(2)))
        erf2 = np.sign(erfarg)*np.vectorize(np.math.erf)(abs(erfarg)/(sigman*np.sqrt(2)))
        IGn_term = exp_term*(erf1 + erf2)
        signal += poisson*w*IGn_term
    return N*signal

def pmtG(xx,N,Q1,s1,mu,w,alpha):
    x = xx
    
    signal = 0
    for n in range(1,3):
        Qn = n*Q1
        sigman = np.sqrt(n)*s1
        poisson = poisson_term(mu,n)
        gauss_term = gaussian(x,Qn,sigman)
        signal += poisson*((1-w)*gauss_term)
    return N*signal

def pmtG_n(xx,N, Q1, s1, mu, w, alpha, n):
    x = xx
    #N, Q1, s1, mu, w, alpha, n = p
    
    Qn = n*Q1
    sigman = np.sqrt(n)*s1
    poisson = poisson_term(mu,n)
    gauss_term = gaussian(x,Qn,sigman)
    signal = poisson*((1-w)*gauss_term)
    
    return N*signal

pmtresponse_model = lmfit.Model(pmtresponse)
params_all = pmtresponse_model.make_params(N=1e5, Q1=200, s1=40, mu=2, w=0.1,alpha=0.001)

pmtbackgroundresponse_model = lmfit.Model(pmtbackgroundresponse)
params_bkg = pmtbackgroundresponse_model.make_params(N=1e5, Q1=200, s1=40, mu=2,w=1, alpha=0.001)

pmtG_model = lmfit.Model(pmtG)
params_2pe = pmtG_model.make_params(N=1e5, Q1=200, s1=40, mu=2, w=0, alpha=0.001) 

pmtG_n_model = lmfit.Model(pmtG_n)
params_pmt = pmtG_n_model.make_params(N=1e5, Q1=200, s1=40, mu=2, w=0, alpha=0.001, n=1) 


In [33]:
def fit_PMT_calibration(cards):                                                                  
    chanrange = 19                                                                                   

    bins=np.arange(0.5,10000.5,1)                                                                    
    bin_centres = bins[:-1]+(bins[1]-bins[0])/2                                                      
                                                                                                     
    result_df = pd.DataFrame([],columns=['card','channel','N','Q1','s1','mu','w','alpha','r2','fit range'])                                                                                              
    
    for card in cards:
        count = 0                                                                                            

        #fig, ax = plt.subplots(4,5, figsize=(24,20),sharex=True,sharey=True)                             
        #fig.suptitle(f"card{card}")                                                                      
        hasdata = 0                                                                                      
        ylim = 1                                                                                       
        for ch in range(chanrange):                                                                      
            this_pmt_charge = charge[(hit_mpmt_card_ids==card)&(hit_pmt_channel_ids==ch)&good]     
            if len(this_pmt_charge) > 1:                                                                 
                hasdata = 1                                                                              
                                                                                                         
                # Histogram                                                                              
                hist,bins = np.histogram(this_pmt_charge, bins=bins)                           
                                                                                                     
                # Initialize parameters                                                                  
                params_0 = pmtresponse_model.make_params(N=1e5, Q1=200, s1=40, mu=2, w=0.1,alpha=0.001)  
                params_0['w'].set(min=0, max=1)   # w must be between 0 and 1                            
                                                                                                     
                try:                                                                                     
                    # Try fitting the data                                                               
                    r2 = 0                                                                               
                    range_upper_lim = 1500                                                               
                    count = 0                                                                            
                    while r2 < 0.992 and count < 4:   
                        y = hist[50:range_upper_lim]
                        x = bin_centres[50:range_upper_lim]
    
                        result = pmtresponse_model.fit(y,params_0,xx=x)

                        # Check if fit converged                                                         
                        if result.success:
                            # Prepare fit label                                                          
                            result_N = result.best_values["N"]
                            result_Q1 = result.best_values["Q1"]
                            result_s1 = result.best_values["s1"]
                            result_mu = result.best_values["mu"]
                            result_w = result.best_values["w"]
                            result_alpha = result.best_values["alpha"]
                            r2 = get_R_square(y, result)

                        else:
                            print(f"Fit failed for channel {ch}")

                        range_upper_lim +=500
                        count = count +1
                        
                    result_df.loc[len(result_df)] = [card,ch,result_N,result_Q1,result_s1,result_mu,result_w,result_alpha,r2,range_upper_lim]

                except Exception as e:
                    # Catch any errors and continue                                                      
                    print(f"Error fitting channel {ch}: {e}")

    return result_df

In [34]:
def get_R_square(y,Result):
    residuals = Result.residual
    ss_res = np.sum(residuals**2)  # Residual sum of squares
    ss_tot = np.sum((y - np.mean(y))**2)  # Total sum of squares
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

In [37]:
charge = hit_pmt_charges.astype(np.float32)
results = fit_PMT_calibration(set(hit_mpmt_card_ids))

Error fitting channel 3: The model function generated NaN values and the fit aborted! Please check your model function and/or set boundaries on parameters where applicable. In cases like this, using "nan_policy='omit'" will probably not work.
Fit failed for channel 2
Error fitting channel 11: The model function generated NaN values and the fit aborted! Please check your model function and/or set boundaries on parameters where applicable. In cases like this, using "nan_policy='omit'" will probably not work.


In [38]:
results.to_pickle("charge_fits")